In [2]:
import pandas as pd

In [3]:
fnames = [
    # "raw/origin_destination_bus_202409.csv",
    # "raw/origin_destination_train_202409.csv",
    "raw/transport_node_bus_202407.csv",
    "raw/transport_node_bus_202408.csv",
    "raw/transport_node_bus_202409.csv",
    # "raw/transport_node_train_202409.csv",
]

# Train data
For train stations that have multiple codes, such as Outram Park and Dhoby Ghaut, we create a duplicate row with a different train code to facilitate joining with other data.

In [9]:
train_node_df = pd.read_csv(fnames[3])
train_node_df.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
0,2024-09,WEEKDAY,10,TRAIN,NS15,14564,10092
1,2024-09,WEEKDAY,14,TRAIN,NE12/CC13,58430,57054
2,2024-09,WEEKDAY,14,TRAIN,NS5,22584,23041
3,2024-09,WEEKDAY,19,TRAIN,SW2,6147,11089
4,2024-09,WEEKDAY,12,TRAIN,BP11,3109,2027


In [10]:
train_node_df.dtypes

YEAR_MONTH              object
DAY_TYPE                object
TIME_PER_HOUR            int64
PT_TYPE                 object
PT_CODE                 object
TOTAL_TAP_IN_VOLUME      int64
TOTAL_TAP_OUT_VOLUME     int64
dtype: object

In [13]:
# Split the PT_CODE column values that contain a slash and expand those rows
train_node_df = train_node_df.assign(PT_CODE=train_node_df['PT_CODE'].str.split('/')).explode('PT_CODE').reset_index(drop=True)
train_node_df.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
0,2024-09,WEEKDAY,10,TRAIN,NS15,14564,10092
1,2024-09,WEEKDAY,14,TRAIN,NE12,58430,57054
2,2024-09,WEEKDAY,14,TRAIN,CC13,58430,57054
3,2024-09,WEEKDAY,14,TRAIN,NS5,22584,23041
4,2024-09,WEEKDAY,19,TRAIN,SW2,6147,11089


In [30]:
train_origin_dest_df = pd.read_csv(fnames[1])
print(train_origin_dest_df.shape)
train_origin_dest_df.head()

(867011, 7)


,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2024-09,WEEKDAY,13,TRAIN,DT13,EW14/NS26,161
1,2024-09,WEEKDAY,12,TRAIN,EW28,EW27,3358
2,2024-09,WEEKDAY,8,TRAIN,NS11,EW23,1484
3,2024-09,WEEKDAY,22,TRAIN,CE1/DT16,EW10,675
4,2024-09,WEEKDAY,22,TRAIN,NE14,PE1,56


In [31]:
train_origin_dest_df = train_origin_dest_df.assign(ORIGIN_PT_CODE=train_origin_dest_df['ORIGIN_PT_CODE'].str.split('/')).explode('ORIGIN_PT_CODE').reset_index(drop=True)
train_origin_dest_df = train_origin_dest_df.assign(DESTINATION_PT_CODE=train_origin_dest_df['DESTINATION_PT_CODE'].str.split('/')).explode('DESTINATION_PT_CODE').reset_index(drop=True)

In [32]:
print(train_origin_dest_df.shape)
train_origin_dest_df.head()

(1267462, 7)


,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2024-09,WEEKDAY,13,TRAIN,DT13,EW14,161
1,2024-09,WEEKDAY,13,TRAIN,DT13,NS26,161
2,2024-09,WEEKDAY,12,TRAIN,EW28,EW27,3358
3,2024-09,WEEKDAY,8,TRAIN,NS11,EW23,1484
4,2024-09,WEEKDAY,22,TRAIN,CE1,EW10,675


In [36]:
train_node_df.to_csv('cleaned/TrainRideVolume202409.csv', index=False)
train_origin_dest_df.to_csv('cleaned/TrainRideVolumeOrigDest202409.csv', index=False)

# Bus data

In [4]:
# Load all bus node files and concatenate them into a single dataframe
bus_node_df = pd.concat([pd.read_csv(fname) for fname in fnames], ignore_index=True)
print(bus_node_df.shape)
bus_node_df.head()

(598518, 7)


,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
0,2024-07,WEEKDAY,19.0,BUS,46359,279,138
1,2024-07,WEEKDAY,6.0,BUS,64551,570,26
2,2024-07,WEEKDAY,9.0,BUS,92099,2009,866
3,2024-07,WEEKDAY,6.0,BUS,12201,51,1405
4,2024-07,WEEKDAY,17.0,BUS,77101,94,110


In [35]:
bus_origin_dest_df = pd.read_csv(fnames[0])
print(bus_origin_dest_df.shape)
bus_origin_dest_df.head()

(5721503, 7)


,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2024-09,WEEKENDS/HOLIDAY,20,BUS,46009,46249,586
1,2024-09,WEEKENDS/HOLIDAY,19,BUS,59091,59419,54
2,2024-09,WEEKENDS/HOLIDAY,23,BUS,54181,54201,72
3,2024-09,WEEKENDS/HOLIDAY,14,BUS,61039,61111,256
4,2024-09,WEEKENDS/HOLIDAY,14,BUS,44629,44009,902


In [7]:
bus_node_df.to_csv('cleaned/BusRideVolume_2024_070809.csv', index=False)
# bus_origin_dest_df.to_csv('cleaned/BusRideVolumeOrigDest202409.csv', index=False)